<a href="https://colab.research.google.com/github/rebeccali/xflr5_parser/blob/master/Foil_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

This is a simple analysis to find the maximum pitch up moment for a given alpha. We integrate this over time in order to find out how long it takes to do this.

We have some $C_M(alpha)$ and $C_L(\alpha)$, and can use this plus max thrust to derive 
pitch acceleration or $\ddot{\alpha} = f(\alpha)$. Then we simply integrate this to get $\alpha(t)$, $\alpha(0) = 0$, $\alpha(T) > 90^o$

$\ddot{\alpha} = -C_L(\alpha) q S r_L + C_M(\alpha) q S c + r_T T sin\theta$



* S is wing area
* c is chord length
* $r_L$ is distance from center of mass to quarter chord
* $r_T$ is distance from center of mass to thrust pivot
* T is thrust
* $\theta$ is propellor angle, postive being pointed up


The numeric integration we need to perform is:

$\alpha(T) = \int\int\ddot{\alpha} dt dt $

The limits of integration [0,T] don't matter as long as T is long enough such that we see how long it takes to hit 90 degrees. 

